# Day 12 - claude

In [3]:
def read_input(filename):
    with open(filename, 'r') as f:
        return [list(line.strip()) for line in f.readlines()]

def get_neighbors(x, y, grid):
    neighbors = []
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    rows, cols = len(grid), len(grid[0])
    
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < rows and 0 <= new_y < cols:
            neighbors.append((new_x, new_y))
    return neighbors

def find_region(start_x, start_y, grid, visited):
    plant_type = grid[start_x][start_y]
    region = set()
    stack = [(start_x, start_y)]
    
    while stack:
        x, y = stack.pop()
        if (x, y) in visited or grid[x][y] != plant_type:
            continue
            
        visited.add((x, y))
        region.add((x, y))
        
        for next_x, next_y in get_neighbors(x, y, grid):
            if grid[next_x][next_y] == plant_type and (next_x, next_y) not in visited:
                stack.append((next_x, next_y))
                
    return region, plant_type

def calculate_perimeter(region, grid):
    perimeter = 0
    total_sides = len(region) * 4  # Each plot has 4 sides
    shared_sides = 0
    
    # Count shared sides between plots in the same region
    for x, y in region:
        for nx, ny in get_neighbors(x, y, grid):
            if (nx, ny) in region:
                shared_sides += 1
    
    # Perimeter is total sides minus twice the shared sides
    # (because each shared side was counted twice)
    perimeter = total_sides - shared_sides
    return perimeter

def solve_garden_plots(grid):
    rows, cols = len(grid), len(grid[0])
    visited = set()
    total_price = 0
    
    # For debugging
    prices = []
    
    for i in range(rows):
        for j in range(cols):
            if (i, j) not in visited:
                region, plant_type = find_region(i, j, grid, visited)
                area = len(region)
                perimeter = calculate_perimeter(region, grid)
                price = area * perimeter
                total_price += price
                
                # Debug output
                prices.append((plant_type, area, perimeter, price))
    
    # Print debug information
    print("\nRegion details:")
    for plant_type, area, perimeter, price in sorted(prices):
        print(f"Type {plant_type}: area={area}, perimeter={perimeter}, price={price}")
                
    return total_price

def main():
    grid = read_input('input.txt')
    result = solve_garden_plots(grid)
    print(f"\nTotal price of fencing: {result}")

if __name__ == "__main__":
    main()


Region details:
Type A: area=1, perimeter=4, price=4
Type A: area=1, perimeter=4, price=4
Type A: area=1, perimeter=4, price=4
Type A: area=1, perimeter=4, price=4
Type A: area=1, perimeter=4, price=4
Type A: area=1, perimeter=4, price=4
Type A: area=1, perimeter=4, price=4
Type A: area=2, perimeter=6, price=12
Type A: area=2, perimeter=6, price=12
Type A: area=3, perimeter=8, price=24
Type A: area=3, perimeter=8, price=24
Type A: area=4, perimeter=8, price=32
Type A: area=5, perimeter=10, price=50
Type A: area=6, perimeter=12, price=72
Type A: area=7, perimeter=16, price=112
Type A: area=12, perimeter=20, price=240
Type A: area=26, perimeter=38, price=988
Type A: area=38, perimeter=34, price=1292
Type A: area=67, perimeter=48, price=3216
Type A: area=72, perimeter=62, price=4464
Type A: area=83, perimeter=66, price=5478
Type A: area=104, perimeter=78, price=8112
Type A: area=197, perimeter=142, price=27974
Type B: area=1, perimeter=4, price=4
Type B: area=1, perimeter=4, price=4
Type

## Part 2

In [8]:
def read_input(filename):
    with open(filename, 'r') as f:
        return [list(line.strip()) for line in f.readlines()]

def get_neighbors(x, y, grid):
    neighbors = []
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    rows, cols = len(grid), len(grid[0])
    
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < rows and 0 <= new_y < cols:
            neighbors.append((new_x, new_y))
    return neighbors

def find_region(start_x, start_y, grid, visited):
    plant_type = grid[start_x][start_y]
    region = set()
    stack = [(start_x, start_y)]
    
    while stack:
        x, y = stack.pop()
        if (x, y) in visited or grid[x][y] != plant_type:
            continue
            
        visited.add((x, y))
        region.add((x, y))
        
        for next_x, next_y in get_neighbors(x, y, grid):
            if grid[next_x][next_y] == plant_type and (next_x, next_y) not in visited:
                stack.append((next_x, next_y))
                
    return region, plant_type

def analyze_boundary(region, grid):
    # Store edge segments between cells
    edges = set()
    # Store all the corner points where edges meet
    corners = set()
    
    for x, y in region:
        # Check all four sides of this cell
        # Right side
        if (x, y+1) not in region:
            edges.add(('V', x, y+1))
            corners.add((x, y+1))
            corners.add((x+1, y+1))
        # Bottom side
        if (x+1, y) not in region:
            edges.add(('H', x+1, y))
            corners.add((x+1, y))
            corners.add((x+1, y+1))
        # Left side
        if (x, y-1) not in region:
            edges.add(('V', x, y))
            corners.add((x, y))
            corners.add((x+1, y))
        # Top side
        if (x-1, y) not in region:
            edges.add(('H', x, y))
            corners.add((x, y))
            corners.add((x, y+1))
    
    # Count the number of distinct edges by following the boundary
    sides = 0
    current = corners.pop() if corners else None
    start = current
    used_edges = set()
    
    while current and corners:
        # Look for connected corners through unused edges
        found = False
        for edge in edges:
            if edge in used_edges:
                continue
                
            edge_type, ex, ey = edge
            if edge_type == 'V':
                if (ex, ey) == current or (ex+1, ey) == current:
                    next_corner = (ex+1, ey) if (ex, ey) == current else (ex, ey)
                    if next_corner in corners:
                        sides += 1
                        used_edges.add(edge)
                        corners.remove(next_corner)
                        current = next_corner
                        found = True
                        break
            else:  # horizontal edge
                if (ex, ey) == current or (ex, ey+1) == current:
                    next_corner = (ex, ey+1) if (ex, ey) == current else (ex, ey)
                    if next_corner in corners:
                        sides += 1
                        used_edges.add(edge)
                        corners.remove(next_corner)
                        current = next_corner
                        found = True
                        break
        
        if not found:
            # If no connected edge found, start new path with remaining corner
            if corners:
                current = corners.pop()
                start = current
    
    return sides

def solve_garden_plots(grid):
    rows, cols = len(grid), len(grid[0])
    visited = set()
    total_price = 0
    
    # For debugging
    prices = []
    
    for i in range(rows):
        for j in range(cols):
            if (i, j) not in visited:
                region, plant_type = find_region(i, j, grid, visited)
                area = len(region)
                sides = analyze_boundary(region, grid)
                price = area * sides
                total_price += price
                
                # Debug output
                prices.append((plant_type, area, sides, price))
    
    # Print debug information
    print("\nRegion details:")
    for plant_type, area, sides, price in sorted(prices):
        print(f"Type {plant_type}: area={area}, sides={sides}, price={price}")
                
    return total_price

def main():
    grid = read_input('input.txt')
    result = solve_garden_plots(grid)
    print(f"\nTotal price of fencing: {result}")

if __name__ == "__main__":
    main()


Region details:
Type A: area=1, sides=3, price=3
Type A: area=1, sides=3, price=3
Type A: area=1, sides=3, price=3
Type A: area=1, sides=3, price=3
Type A: area=1, sides=3, price=3
Type A: area=1, sides=3, price=3
Type A: area=1, sides=3, price=3
Type A: area=2, sides=5, price=10
Type A: area=2, sides=5, price=10
Type A: area=3, sides=7, price=21
Type A: area=3, sides=7, price=21
Type A: area=4, sides=7, price=28
Type A: area=5, sides=9, price=45
Type A: area=6, sides=11, price=66
Type A: area=7, sides=15, price=105
Type A: area=12, sides=19, price=228
Type A: area=26, sides=37, price=962
Type A: area=38, sides=33, price=1254
Type A: area=67, sides=47, price=3149
Type A: area=72, sides=61, price=4392
Type A: area=83, sides=65, price=5395
Type A: area=104, sides=74, price=7696
Type A: area=197, sides=137, price=26989
Type B: area=1, sides=3, price=3
Type B: area=1, sides=3, price=3
Type B: area=1, sides=3, price=3
Type B: area=1, sides=3, price=3
Type B: area=1, sides=3, price=3
Type B